## Get Album from Spotify

In [1]:
from tqdm.notebook import tqdm
import pandas as pd
from album_lookup import (
    get_official_album_name,
    guess_album,
    album_candidates,
    guess_album_via_google,
)

In [2]:
# Example test: Try to look up a well-known track and check the album name
artist = "Iron Maiden"
title = "The Trooper"
expected_album = "Piece of Mind"

predicted_album = get_official_album_name(artist, title)
match = guess_album(artist, title)
print(f"Match details: {match}")
print(f"Artist: {artist}")
print(f"Title: {title}")
print(f"Expected Album: {expected_album}")
print(f"Predicted Album: {predicted_album}")

assert predicted_album is not None, "No album found"
assert expected_album.lower() in predicted_album.lower(), "Album lookup failed"

Match details: AlbumMatch(album='Piece of Mind', source='spotify', confidence=0.5777777777777778, album_type='album', raw_album='Piece of Mind (2015 Remaster)', release_date=datetime.datetime(1983, 1, 1, 0, 0), track_id='4OROzZUy6gOWN4UGQVaZMF', track_name='The Trooper - 2015 Remaster', title_score=0.6111111111111112, artist_score=1.0, popularity=74, flags=('reissue',))
Artist: Iron Maiden
Title: The Trooper
Expected Album: Piece of Mind
Predicted Album: Piece of Mind


In [3]:
guess_album("Ensiferum", "In My Sword I Trust")

AlbumMatch(album='Unsung Heroes', source='spotify', confidence=1.0, album_type='album', raw_album='Unsung Heroes (Japan Version)', release_date=datetime.datetime(2012, 1, 1, 0, 0), track_id='7aYO9WJaSC8m7YpT0bM5Ke', track_name='In My Sword I Trust', title_score=1.0, artist_score=1.0, popularity=46, flags=())

In [ ]:
candidates = album_candidates("Iron Maiden", "Powerslave")
candidates

[AlbumMatch(album='Live After Death', source='spotify', confidence=0.55, album_type='album', raw_album='Live After Death (1998 Remaster)', release_date=datetime.datetime(1985, 1, 1, 0, 0), track_id='5iKxoiMFKa8GxDaXGuwApw', track_name='Powerslave - Live at Long Beach Arena; 1998 Remaster', title_score=1.0, artist_score=1.0, popularity=34, flags=('reissue', 'live_track')),
 AlbumMatch(album='Flight 666: The Original Soundtrack', source='spotify', confidence=0.65, album_type='album', raw_album='Flight 666: The Original Soundtrack', release_date=datetime.datetime(2009, 5, 25, 0, 0), track_id='1YeaKKlz9shDs9iHq6qKCH', track_name='Powerslave - Live in San Jose 26 February 2008', title_score=1.0, artist_score=1.0, popularity=31, flags=('live_track',)),
 AlbumMatch(album='The Book of Souls', source='spotify', confidence=0.34999999999999987, album_type='album', raw_album='The Book of Souls: Live Chapter', release_date=datetime.datetime(2017, 11, 17, 0, 0), track_id='2wqnxiwfUr5TqlJlPdM5WR', tr

In [5]:
guess_album("Megadeth", "Symphony of Destruction")

AlbumMatch(album='Countdown To Extinction', source='spotify', confidence=0.85, album_type='album', raw_album='Countdown To Extinction (Deluxe Edition - Remastered)', release_date=datetime.datetime(1992, 6, 14, 0, 0), track_id='5mR858YsHYG761aUqZoGkD', track_name='Symphony Of Destruction - Remastered 2012', title_score=1.0, artist_score=1.0, popularity=69, flags=('reissue',))

In [8]:
melodeath = "NeuralForge/playlists/Melodic Death Metal.csv"
classic_metal = "NeuralForge/playlists/Classic Metal.csv"
power_metal = "NeuralForge/playlists/Power Metal.csv"
folk_metal = "NeuralForge/playlists/Folk Metal.csv"
prog_metal = "NeuralForge/playlists/Prog Metal.csv"

# List of all playlist CSVs
playlist_files = [
    # melodeath,
    classic_metal,
    # power_metal,
    # folk_metal,
    # prog_metal,
]

# Concatenate all playlists into one DataFrame
df_all = pd.concat([pd.read_csv(f) for f in playlist_files], ignore_index=True)

correct = 0
total = 0

for i, row in tqdm(df_all.iterrows(), total=len(df_all)):
    artist = row["Artist"]
    title = row["Title"]
    official_album = row["Album"]
    predicted_album = get_official_album_name(artist, title)
    total += 1
    if predicted_album and str(predicted_album).lower() == str(official_album).lower():
        correct += 1
    else:
        print(f"\nMismatch for '{artist}' - '{title}':")
        print(f"\tOfficial: '{official_album}'")
        print(f"\tPredicted: '{predicted_album}'")

print(f"Correct: {correct}/{total} ({correct / total:.2%})")

  0%|          | 0/155 [00:00<?, ?it/s]


Mismatch for 'Black Sabbath' - 'Heaven and Hell':
	Official: 'Heaven and Hell'
	Predicted: 'Heaven & Hell'

Mismatch for 'Black Sabbath' - 'Iron Man':
	Official: 'Paranoid'
	Predicted: 'The Auburn University Marching Band - Highlights of the 1998 Season'

Mismatch for 'Black Sabbath' - 'N.I.B.':
	Official: 'Black Sabbath'
	Predicted: 'The Ultimate Collection'

Mismatch for 'Black Sabbath' - 'The Wizard':
	Official: 'Black Sabbath'
	Predicted: 'The Ultimate Collection'

Mismatch for 'Black Sabbath' - 'War Pigs':
	Official: 'Paranoid'
	Predicted: 'The Ultimate Collection'

Mismatch for 'Deep Purple' - 'Highway Star':
	Official: 'Machine Head'
	Predicted: 'Live At The Olympia'

Mismatch for 'Deep Purple' - 'Smoke on the Water':
	Official: 'Machine Head'
	Predicted: 'Live At The Olympia'

Mismatch for 'Deep Purple' - 'Speed King':
	Official: 'Deep Purple in Rock'
	Predicted: 'Live At The Olympia'

Mismatch for 'Iron Maiden' - 'Aces High':
	Official: 'Powerslave'
	Predicted: 'Flight 666: T

AlbumMatch(album='Pale Morning Star', source='spotify', confidence=0.9700000000000001, album_type='single', raw_album='Pale Morning Star', release_date=datetime.datetime(2019, 9, 27, 0, 0), track_id='56OUOaYwDT3ErRXFVL8Plu', track_name='Pale Morning Star', title_score=1.0, artist_score=1.0, popularity=11, flags=('type:single',))